In [12]:
import data_saving
import importlib
import funciones
import concat
importlib.reload(funciones)
importlib.reload(concat)
importlib.reload(data_saving)
from concat import concat
from funciones import clean_algabo, clean_furey, clean_canal, scrape_drimel, clean_upalala, clean_teddy

def clean_file(canal,filename, company):
    if company == "Algabo":
        clean_algabo(filename)
        clean_canal(canal,'algabo',mpn_value=company)
        concat('algabo')
    elif company == "Furey":
        clean_furey(filename)
        clean_canal(canal,'furey',mpn_value=company)
        concat('furey')
    elif company == 'Drimel':
        scrape_drimel(filename)
        clean_canal(canal,'drimel',mpn_value=company)
        concat('drimel')
    elif company == 'Upalala':
        clean_upalala(filename)
        clean_canal(canal,'upalala',mpn_value= company)
        concat('upalala')
    elif company == 'Teddy':
        clean_teddy(filename)
        clean_canal(canal, 'teddy', mpn_value = company)
        concat('teddy')
    else:
        raise ValueError("Company not recognized")


In [9]:
# ALGABO
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/algabo.xlsx',
           'Algabo')

Archivo limpio guardado como ./procesados/algabo.xlsx
Archivo limpio guardado como ./procesados/procesadosCanal/algabo.xlsx

En general, los productos aumentaron en un nan%.
De los 310 productos, 0 sufrieron un aumento.

Total de filas en df_result: 310
Filas con similaridad del 100%: 284
Filas de CANAL no encontradas en df2: 15
Filas de df2 no encontradas en CANAL: 11

Archivo CONCATENADO guardado como proveedor/algabo/listos/algabo.xlsx

No se encontraron aumentos excesivos en los productos.
El archivo CSV se ha guardado en: /Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/proveedor/algabo/listos/tiendaNube/algabo.csv


In [14]:
# DRIMEL
clean_file('./sinProcesar/canal.csv',
           ['https://drimel.com.ar/?product_cat=bebes', 'https://drimel.com.ar/?product_cat=cuidado-del-bebe'],
            'Drimel')

Archivo guardado con exito
Archivo limpio guardado como ./procesados/procesadosCanal/drimel.xlsx

En general, los productos aumentaron en un 8.00%.
De los 216 productos, 1 sufrieron un aumento.

Total de filas en df_result: 216
Filas con similaridad del 100%: 143
Filas de CANAL no encontradas en df2: 54
Filas de df2 no encontradas en CANAL: 19

Archivo CONCATENADO guardado como proveedor/drimel/listos/drimel.xlsx

No se encontraron aumentos excesivos en los productos.
El archivo CSV se ha guardado en: /Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/proveedor/drimel/listos/tiendaNube/drimel.csv


In [13]:
# FUREY
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/furey.xlsx',
           'Furey')

Archivo limpio guardado como ./procesados/furey.xlsx
Archivo limpio guardado como ./procesados/procesadosCanal/furey.xlsx


KeyboardInterrupt: 

In [10]:
# UPALALA
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/upalala.xlsx',
           'Upalala')

Archivo limpio guardado como ./procesados/upalala.xlsx
Archivo limpio guardado como ./procesados/procesadosCanal/upalala.xlsx

En general, los productos aumentaron en un nan%.
De los 53 productos, 0 sufrieron un aumento.

Total de filas en df_result: 53
Filas con similaridad del 100%: 29
Filas de CANAL no encontradas en df2: 23
Filas de df2 no encontradas en CANAL: 1

Archivo CONCATENADO guardado como proveedor/upalala/listos/upalala.xlsx

No se encontraron aumentos excesivos en los productos.
El archivo CSV se ha guardado en: /Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/proveedor/upalala/listos/tiendaNube/upalala.csv


In [13]:
# TEDDY
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/teddy.xlsx',
           'Teddy')

Archivo limpio guardado como ./procesados/teddy.xlsx
Imágenes eliminadas del archivo ./procesados/teddy.xlsx
Archivo limpio guardado como ./procesados/procesadosCanal/teddy.xlsx

En general, los productos aumentaron en un 17.22%.
De los 101 productos, 83 sufrieron un aumento.

Total de filas en df_result: 101
Filas con similaridad del 100%: 95
Filas de CANAL no encontradas en df2: 0
Filas de df2 no encontradas en CANAL: 6

Archivo CONCATENADO guardado como proveedor/teddy/listos/teddy.xlsx

Se encontraron 82 productos con un aumento excesivo:
  - Sonajero platitos (SKU: Teddy259) aumentó un 24.00%.
  - CAJITA SONAJERO COCHECITO FANTASÍA (SKU: Teddy5053) aumentó un 24.00%.
  - Mordillo refrigerante color (SKU: Teddy274) aumentó un 23.00%.
  - Mordillo refrigerante (SKU: Teddy273) aumentó un 22.00%.
  - Mordillo refrigerante hipo equilibrista (SKU: Teddy263) aumentó un 22.00%.
  - Mordillo refrigerante Pato Dormilon (SKU: Teddy262) aumentó un 22.00%.
  - Taza con asa y tapa (SKU: Teddy40